# Run one execution

In [ ]:
from zubora_gabora.experiment.aco.aco_experiment_executer import ACOExperimentExecuter
from zubora_gabora.aco.aco_zubora_gabora import ACOZuboraGabora

executer = ACOExperimentExecuter(data_path="data/dataset.csv")

aco = executer.run_single_experiment("1")

print(f"Best fitness: {1.0/aco.best_fitness:.0f}")
ACOZuboraGabora.decode_solution(aco.best_solution)

## Draw Gantt of solution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("darkgrid")

def draw_gantt_tasks_ordered(assignments, schedule):
    """
    Draws a Gantt chart with each row representing a task in the order:
    F1, G1, F2, G2, …, Fn, Gn.
    
    The color of each bar indicates the worker (e.g., Zu, Ga) who performed the task.
    
    Parameters:
      - assignments: list of dictionaries for each sword mapping task ('F' or 'G') to a worker.
      - schedule: list of dictionaries for each sword mapping task ('F' or 'G') to [start, end] times.
    """
    # Define colors for each worker.
    worker_colors = {'Zu': 'tab:blue', 'Ga': 'tab:orange'}
    
    # Build a list of tasks in the desired order.
    tasks_list = []
    for sword_idx, (assign, times) in enumerate(zip(assignments, schedule)):
        # Process tasks in fixed order: F then G.
        for task in ['F', 'G']:
            worker = assign[task]
            start, end = times[task]
            duration = end - start
            label = f"{task}{sword_idx + 1}"
            tasks_list.append({
                'label': label,
                'worker': worker,
                'start': start,
                'duration': duration
            })
    
    # Create the plot.
    fig, ax = plt.subplots(figsize=(10, 6))
    row_height = 8
    row_gap = 10  # vertical spacing between rows
    
    # Plot each task.
    for row_idx, task_item in enumerate(tasks_list):
        y_pos = row_idx * row_gap
        ax.broken_barh(
            [(task_item['start'], task_item['duration'])],
            (y_pos, row_height),
            facecolors=worker_colors.get(task_item['worker'], 'gray'),
            edgecolor='black', alpha=0.8
        )
        # Add the task label inside the bar.
        ax.text(
            task_item['start'] + task_item['duration'] / 2, 
            y_pos + row_height / 2,
            task_item['label'], ha='center', va='center', color='white', fontsize=9
        )
    
    # Set y-ticks and labels.
    ax.set_yticks([i * row_gap + row_height / 2 for i in range(len(tasks_list))])
    ax.set_yticklabels([task['label'] for task in tasks_list])
    ax.set_xlabel("Time")
    ax.set_title("Gantt Chart (Ordered as F1, G1, F2, G2, …)")
    
    # Create a legend mapping colors to workers.
    legend_handles = [plt.Rectangle((0, 0), 1, 1, color=worker_colors[w]) for w in worker_colors]
    ax.legend(legend_handles, worker_colors.keys(), title="Worker", loc='upper right')
    
    plt.tight_layout()
    plt.show()


# Example data (as provided)
assignments = aco.best_solution[0]

_, schedule = aco._simulate_forging(assignments, aco.best_solution[1])

# Draw the ordered Gantt chart.
draw_gantt_tasks_ordered(ACOZuboraGabora._decode_who(assignments), schedule)

## Process Visualization

### Fitness evolution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

fitness = 1/np.array([[ant[1] for ant in trails] for trails in aco.trails_history ])
best_fitness = 1/np.array(aco.best_fitness_history)

fig, axs = plt.subplots(figsize=(5,5))
axs.set_title('Fitness evolution')
axs.set_xlabel('Iterations')
axs.set_ylabel('Fitness')

axs.plot(best_fitness, label='best_high')

median = np.median(fitness, axis=1)
min_array = np.min(fitness, axis=1)
max_array = np.max(fitness, axis=1)
axs.plot(median, label='iterations_high')
axs.fill_between(np.arange(len(median)), min_array, max_array, alpha=0.3, color='orange')

plt.legend()

### Diversity evolution

In [ ]:
population = np.array([[np.array(ant[0]+ant[1]) for ant,_ in trails] for trails in aco.trails_history ])

diversity = np.sum(np.std(population, axis=1), axis=1)
fig, axs = plt.subplots(figsize=(5,5))
axs.set_title('Diversity evolution')
axs.set_xlabel('Iterations')
axs.set_ylabel('Diversity')
axs.plot(diversity, color='orange')

# Repeat experiment several times

In [ ]:
from zubora_gabora.experiment.aco.aco_experiment_executer import ACOExperimentExecuter

executer = ACOExperimentExecuter(data_path="data/dataset.csv")

data = executer.run_repeated_experiment("1", n_repeat=10)

print(f"Best run {data['fitness'].idxmax()}: fitness {data['fitness'].max():.0f} in {data['n_evaluations'].iloc[data['fitness'].idxmax()]} fitness evaluations")
print(f"Mean fitness: {data['fitness'].mean():.2f} | Std {data['fitness'].std():.2f}")
print(f"Mean number of fitness evaluations: {data['n_evaluations'].mean()}")

# Run all experiments

In [ ]:
from zubora_gabora.experiment.aco.aco_experiment_executer import ACOExperimentExecuter

executer = ACOExperimentExecuter(data_path="data/dataset.csv")
executer.run_all_experiments("experiments/aco/", n_repeat=31)